In [74]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats.mstats import winsorize
import statsmodels.api as sm

資料整理

In [71]:
# 1. 讀取資料
print("讀取資料中...")
df_sga = pd.read_csv('../data/processed/main_sga_v2_result.csv')
df_capex = pd.read_csv('../data/raw/capex.csv')
df_industry = pd.read_csv('../data/raw/industry.csv')

# 讀取市值資料 (用於計算 Size)
df_mv = pd.read_csv('../data/raw/industry&return.csv', 
                    dtype={'年月': str},
                    usecols=['證券代碼', '年月', '市值(百萬元)'])

# 2. 資料清洗與合併準備
df_sga['Company_ID'] = df_sga['Company_ID'].astype(str)

df_capex.columns = df_capex.columns.str.strip()
df_capex['Company_ID'] = df_capex['公司'].astype(str).str.split().str[0]
df_capex['Year'] = pd.to_datetime(df_capex['年月']).dt.year

# 處理數值欄位
df_capex['PPE_Capex'] = pd.to_numeric(df_capex['購置不動產廠房設備（含預付）－CFI'].astype(str).str.replace(',', ''), errors='coerce').abs()
df_capex['Intan_Capex'] = pd.to_numeric(df_capex['無形資產（增加）減少－CFI'].astype(str).str.replace(',', ''), errors='coerce').abs()
df_capex['Tobins_Q'] = pd.to_numeric(df_capex['Tobins Q'], errors='coerce')
# 新增: 處理負債總額
if '負債總額' in df_capex.columns:
    df_capex['Total_Liabilities'] = pd.to_numeric(df_capex['負債總額'].astype(str).str.replace(',', ''), errors='coerce')
else:
    df_capex['Total_Liabilities'] = np.nan

# 準備控制變數與產業代碼
df_industry.columns = df_industry.columns.str.strip()
df_industry['Company_ID'] = df_industry['公司'].astype(str).str.split().str[0]
df_industry['Year'] = pd.to_datetime(df_industry['年月']).dt.year
df_industry['Net_Income'] = pd.to_numeric(df_industry['歸屬母公司淨利（損）'].astype(str).str.replace(',', ''), errors='coerce')
df_industry['Total_Assets'] = pd.to_numeric(df_industry['資產總額'].astype(str).str.replace(',', ''), errors='coerce')
# 處理產業代碼
df_industry['Industry_Group'] = df_industry['TEJ產業_代碼'].astype(str).str[:3]

df_industry = df_industry.sort_values(['Company_ID', 'Year'])
df_industry['Lag_Net_Income'] = df_industry.groupby('Company_ID')['Net_Income'].shift(1)
df_industry['Current_Earnings_Growth'] = (df_industry['Net_Income'] - df_industry['Lag_Net_Income']) / df_industry['Lag_Net_Income'].abs()
df_industry['Current_Earnings_Growth'] = df_industry['Current_Earnings_Growth'].replace([np.inf, -np.inf], np.nan)

# 準備市值資料
df_mv['Company_ID'] = df_mv['證券代碼'].astype(str).str.split().str[0]
df_mv['Date'] = pd.to_datetime(df_mv['年月'], format='%Y%m')
df_mv['Year'] = df_mv['Date'].dt.year
df_mv['MV'] = pd.to_numeric(df_mv['市值(百萬元)'].astype(str).str.replace(',', ''), errors='coerce')
# 取每家公司每年的平均市值或年底市值
df_mv_year = df_mv.sort_values('Date').groupby(['Company_ID', 'Year'])['MV'].last().reset_index()

# 3. 合併所有資料
# 先合併 Capex (含負債總額)
df_merged = pd.merge(df_sga, df_capex[['Company_ID', 'Year', 'PPE_Capex', 'Intan_Capex', 'Tobins_Q', 'Total_Liabilities']], 
                     on=['Company_ID', 'Year'], how='inner')

# 合併 Industry (獲利成長, 資產總額等) - Industry_Group 已在 df_sga 中，無需重複合併
df_merged = pd.merge(df_merged, df_industry[['Company_ID', 'Year', 'Net_Income', 'Current_Earnings_Growth', 'Total_Assets']], 
                     on=['Company_ID', 'Year'], how='left')


# 合併市值 (MV)
df_merged = pd.merge(df_merged, df_mv_year, on=['Company_ID', 'Year'], how='left')

# 4. 變數標準化 (Scaling by Assets)
df_merged['Scaled_RD'] = df_merged['研究發展費'] / df_merged['Avg_Total_Assets']
df_merged['Scaled_PPE_Capex'] = df_merged['PPE_Capex'] / df_merged['Avg_Total_Assets']
df_merged['Scaled_Intan_Capex'] = df_merged['Intan_Capex'] / df_merged['Avg_Total_Assets']
df_merged['Scaled_Earnings'] = df_merged['Net_Income'] / df_merged['Avg_Total_Assets']
df_merged['Scaled_Intan_Capex'] = df_merged['Scaled_Intan_Capex'].fillna(0)

# 5. 建構控制變數
df_merged['Size'] = np.log(df_merged['MV'])
df_merged['Leverage'] = df_merged['Total_Liabilities'] / df_merged['Total_Assets']
df_merged['Loss_Dummy'] = (df_merged['Net_Income'] < 0).astype(int)
df_merged['Current_Earnings_Growth'] = df_merged['Current_Earnings_Growth'].fillna(0)

# 6. 建構應變數: Future Earnings Change
df_merged = df_merged.sort_values(by=['Company_ID', 'Year'])
for i in [1, 2, 3]:
    df_merged[f'Earnings_t{i}'] = df_merged.groupby('Company_ID')['Scaled_Earnings'].shift(-i)

df_merged['Avg_Future_Earnings'] = df_merged[[f'Earnings_t{i}' for i in [1, 2, 3]]].mean(axis=1)
df_merged['Change_In_Earnings'] = df_merged['Avg_Future_Earnings'] - df_merged['Scaled_Earnings']

# 7. 準備迴歸資料
cols_to_use = ['Change_In_Earnings', 'Tobins_Q', 'Scaled_RD', 'Investment_MainSGA', 
               'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
               'Size', 'Current_Earnings_Growth', 'Loss_Dummy', 'Leverage', 'Company_ID', 'Year', 'Industry_Group']

df_reg = df_merged.dropna(subset=cols_to_use).copy()

# === Step A: Winsorization (1% & 99%) on RAW values ===
print("正在進行 Winsorization (1%, 99%)...")
vars_to_winsorize = ['Change_In_Earnings', 'Tobins_Q', 'Scaled_RD', 'Investment_MainSGA', 
                     'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
                     'Size', 'Current_Earnings_Growth', 'Leverage']

for col in vars_to_winsorize:
    df_reg[col] = winsorize(df_reg[col], limits=[0.01, 0.01])

# === Step B: Industry-Year Standardization (Z-Score) ===
print("正在進行 Industry-Year Standardization...")
# 定義需要標準化的投資變數
invest_vars = ['Scaled_RD', 'Investment_MainSGA', 'Scaled_PPE_Capex', 'Scaled_Intan_Capex']

for col in invest_vars:
    new_col = f'Std_{col}'
    # 分組計算 Z-Score: (x - mean) / std
    # 使用 transform 以保持原 index
    grouped = df_reg.groupby(['Industry_Group', 'Year'])[col]
    mean = grouped.transform('mean')
    std = grouped.transform('std')
    
    df_reg[new_col] = (df_reg[col] - mean) / std
    
    # 處理 std 為 0 或 NaN 的情況 (若該產業該年只有一家公司或數值都一樣)
    df_reg[new_col] = df_reg[new_col].fillna(0)

# 檢查一下標準化後的統計量
print("\n標準化變數統計量 (前5筆):")
print(df_reg[['Std_Scaled_RD', 'Std_Investment_MainSGA', 'Std_Scaled_PPE_Capex']].describe())

# === 關鍵修正：將 Company_ID 轉為數值編碼 ===
df_reg['Company_Code'] = df_reg['Company_ID'].astype('category').cat.codes
df_reg['Year_Code'] = df_reg['Year'].astype(int)

# 建立群聚變數矩陣 (N x 2)
groups = df_reg[['Company_Code', 'Year_Code']].values

print(f"\n分析樣本數: {len(df_reg)}")

讀取資料中...
正在進行 Winsorization (1%, 99%)...
正在進行 Industry-Year Standardization...

標準化變數統計量 (前5筆):
       Std_Scaled_RD  Std_Investment_MainSGA  Std_Scaled_PPE_Capex
count   1.653800e+04            16538.000000          1.653800e+04
mean    1.374856e-17                0.000000          9.452135e-18
std     9.865149e-01                0.995181          9.951810e-01
min    -1.243476e+00               -3.126351         -1.774109e+00
25%    -6.199068e-01               -0.585781         -6.405773e-01
50%    -2.674951e-01               -0.194757         -3.397323e-01
75%     1.934223e-01                0.350132          2.437638e-01
max     9.814959e+00                8.378444          9.310124e+00

分析樣本數: 16538


##
Future Earnings Change Raw

In [65]:
# Model A1: Future Earnings (Raw Values)
print("\n=== Equation 5 (A1): Future Earnings Change (Raw Values) ===")
formula_a1 = 'Change_In_Earnings ~ Scaled_RD + Investment_MainSGA + Scaled_PPE_Capex + Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_a1 = smf.ols(formula_a1, data=df_reg)
res_a1 = model_a1.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_a1.summary())


=== Equation 5 (A1): Future Earnings Change (Raw Values) ===
                            OLS Regression Results                            
Dep. Variable:     Change_In_Earnings   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     146.6
Date:                Fri, 12 Dec 2025   Prob (F-statistic):           1.29e-08
Time:                        03:33:04   Log-Likelihood:                 23715.
No. Observations:               16538   AIC:                        -4.741e+04
Df Residuals:                   16529   BIC:                        -4.734e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------

In [66]:
# Model A2: Future Earnings (Standardized Values)
print("\n=== Equation 5 (A2): Future Earnings Change (Standardized Inv) ===")
formula_a2 = 'Change_In_Earnings ~ Std_Scaled_RD + Std_Investment_MainSGA + Std_Scaled_PPE_Capex + Std_Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_a2 = smf.ols(formula_a2, data=df_reg)
res_a2 = model_a2.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_a2.summary())


=== Equation 5 (A2): Future Earnings Change (Standardized Inv) ===
                            OLS Regression Results                            
Dep. Variable:     Change_In_Earnings   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     134.0
Date:                Fri, 12 Dec 2025   Prob (F-statistic):           1.92e-08
Time:                        03:33:04   Log-Likelihood:                 23703.
No. Observations:               16538   AIC:                        -4.739e+04
Df Residuals:                   16529   BIC:                        -4.732e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------

Tobin's Q

In [67]:
# Model B1: Tobin's Q (Raw Values)
print("\n=== Equation 5 (B1): Tobin\'s Q (Raw Values) ===")
formula_b1 = 'Tobins_Q ~ Scaled_RD + Investment_MainSGA + Scaled_PPE_Capex + Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_b1 = smf.ols(formula_b1, data=df_reg)
res_b1 = model_b1.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_b1.summary())


=== Equation 5 (B1): Tobin's Q (Raw Values) ===
                            OLS Regression Results                            
Dep. Variable:               Tobins_Q   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     50.02
Date:                Fri, 12 Dec 2025   Prob (F-statistic):           1.47e-06
Time:                        03:33:04   Log-Likelihood:                -21182.
No. Observations:               16538   AIC:                         4.238e+04
Df Residuals:                   16529   BIC:                         4.245e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------

In [68]:
# Model B2: Tobin's Q (Standardized Values)
print("\n=== Equation 5 (B2): Tobin\'s Q (Standardized Inv) ===")
formula_b2 = 'Tobins_Q ~ Std_Scaled_RD + Std_Investment_MainSGA + Std_Scaled_PPE_Capex + Std_Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_b2 = smf.ols(formula_b2, data=df_reg)
res_b2 = model_b2.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_b2.summary())


=== Equation 5 (B2): Tobin's Q (Standardized Inv) ===
                            OLS Regression Results                            
Dep. Variable:               Tobins_Q   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     48.53
Date:                Fri, 12 Dec 2025   Prob (F-statistic):           1.68e-06
Time:                        03:33:04   Log-Likelihood:                -21472.
No. Observations:               16538   AIC:                         4.296e+04
Df Residuals:                   16529   BIC:                         4.303e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------

#整理係數比較 (Table 7 Style)

In [69]:
# 整理係數比較 (Table 7 Style)
print("\n" + "="*100)
print("【Table 7 Replication】: Future Benefits of Investment Outlays")
print("="*100)
print(f"{'':<25} | {'Future Earnings (Raw)':<22} | {'Future Earnings (Std)':<22} | {'Tobin Q (Raw)':<20} | {'Tobin Q (Std)':<20}")
print("-" * 115)

rows = [
    ('R&D', 'Scaled_RD', 'Std_Scaled_RD'),
    ('MainSG&A Inv', 'Investment_MainSGA', 'Std_Investment_MainSGA'),
    ('PPE Capex', 'Scaled_PPE_Capex', 'Std_Scaled_PPE_Capex'),
    ('Intan Capex', 'Scaled_Intan_Capex', 'Std_Scaled_Intan_Capex'),
    ('Leverage', 'Leverage', 'Leverage') # Leverage is typically raw in both or standardized in both? Paper standardizes only investment variables. We keep Leverage raw in both or standardize? Let's check raw leverage coefficient in standardized model. Wait, formula_a2 uses 'Leverage' (raw). That's fine.
]

for label, raw_var, std_var in rows:
    # 判斷變數是否存在於模型中 (Leverage 在 Std 模型中是 Raw Leverage)
    coef_a1 = res_a1.params[raw_var]
    t_a1 = res_a1.tvalues[raw_var]
    
    # 對於標準化模型，Leverage 變數名稱還是 'Leverage'
    var_a2 = std_var if std_var in res_a2.params else raw_var
    coef_a2 = res_a2.params[var_a2]
    t_a2 = res_a2.tvalues[var_a2]
    
    coef_b1 = res_b1.params[raw_var]
    t_b1 = res_b1.tvalues[raw_var]
    
    var_b2 = std_var if std_var in res_b2.params else raw_var
    coef_b2 = res_b2.params[var_b2]
    t_b2 = res_b2.tvalues[var_b2]
    
    # 標記顯著性
    def get_sig(p):
        return "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.1 else ""
        
    sig_a1 = get_sig(res_a1.pvalues[raw_var])
    sig_a2 = get_sig(res_a2.pvalues[var_a2])
    sig_b1 = get_sig(res_b1.pvalues[raw_var])
    sig_b2 = get_sig(res_b2.pvalues[var_b2])

    print(f"{label:<25} | {coef_a1:>8.4f}{sig_a1:<3} (t={t_a1:.2f}) | {coef_a2:>8.4f}{sig_a2:<3} (t={t_a2:.2f}) | {coef_b1:>8.4f}{sig_b1:<3} (t={t_b1:.2f}) | {coef_b2:>8.4f}{sig_b2:<3} (t={t_b2:.2f})")

print("-" * 115)
print("註: 括號內為 t-value (Cluster-Robust)。 *** p<0.01, ** p<0.05, * p<0.1")


【Table 7 Replication】: Future Benefits of Investment Outlays
                          | Future Earnings (Raw)  | Future Earnings (Std)  | Tobin Q (Raw)        | Tobin Q (Std)       
-------------------------------------------------------------------------------------------------------------------
R&D                       |   0.0571    (t=1.12) |   0.0014    (t=0.83) |   6.6835*** (t=9.45) |   0.2384*** (t=9.48)
MainSG&A Inv              |   0.0034    (t=0.35) |   0.0010    (t=1.34) |   1.7272*** (t=6.52) |   0.1552*** (t=7.35)
PPE Capex                 |  -0.1020*** (t=-7.02) |  -0.0047*** (t=-7.02) |   2.2694*** (t=7.06) |   0.0907*** (t=6.19)
Intan Capex               |  -0.2582*   (t=-1.73) |  -0.0010    (t=-1.64) |   6.9652*   (t=1.79) |   0.0435*** (t=2.59)
Leverage                  |   0.0323*** (t=5.97) |   0.0312*** (t=5.07) |  -1.3076*** (t=-12.47) |  -1.5490*** (t=-13.17)
------------------------------------------------------------------------------------------------------

###
投組分析

In [70]:
# (A) 財報資料 (含 Investment_MainSGA)
df_sga = pd.read_csv('../data/processed/main_sga_v2_result.csv')
df_sga['Company_ID'] = df_sga['Company_ID'].astype(str)
# 只需要這三個欄位
df_rank = df_sga[['Company_ID', 'Year', 'Investment_MainSGA']].dropna()

# (B) 報酬率資料
# 由於檔案較大，我們指定 dtype 以節省記憶體
df_ret = pd.read_csv('../data/raw/industry&return.csv', 
                     dtype={'年月': str}, # 年月先讀成字串比較好處理
                     usecols=['證券代碼', '年月', '報酬率_月', '市值(百萬元)'])

# 清洗報酬率資料
df_ret['Company_ID'] = df_ret['證券代碼'].astype(str).str.split().str[0]
df_ret['Return'] = pd.to_numeric(df_ret['報酬率_月'], errors='coerce')
# 轉換日期
df_ret['Date'] = pd.to_datetime(df_ret['年月'], format='%Y%m')
df_ret['Ret_Year'] = df_ret['Date'].dt.year
df_ret['Ret_Month'] = df_ret['Date'].dt.month

# 移除沒有報酬率的資料
df_ret = df_ret.dropna(subset=['Return'])

print(f"報酬率資料筆數: {len(df_ret)}")

# 2. 建立投資組合 (Portfolio Formation)
print("正在建構投資組合...")

# 儲存每家公司在每個年度的排名分組
# 對應邏輯: Financial Year T -> 影響 Return Period: (T+1).05 ~ (T+2).04
portfolio_assignments = []

years = sorted(df_rank['Year'].unique())
for year in years:
    # 取得該年度所有公司的數據
    current_year_data = df_rank[df_rank['Year'] == year].copy()
    
    # 根據 Investment_MainSGA 進行分組 (Q1=Low, Q5=High)
    # qcut 可能會因為重複值報錯，如果遇到重複值改用 rank pct
    try:
        current_year_data['Rank'] = pd.qcut(current_year_data['Investment_MainSGA'], 5, labels=[1, 2, 3, 4, 5])
    except:
        # 如果數據太集中導致無法切分，改用百分比排名
        current_year_data['Pct'] = current_year_data['Investment_MainSGA'].rank(pct=True)
        current_year_data['Rank'] = pd.cut(current_year_data['Pct'], bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=[1, 2, 3, 4, 5])
    
    # 定義這個排名適用的報酬率期間: Year+1 的 5月 到 Year+2 的 4月
    start_date = pd.Timestamp(f"{year+1}-05-01")
    end_date = pd.Timestamp(f"{year+2}-04-30")
    
    # 將分組資訊展開到該期間的每一個月份 (這一步是為了後續方便 merge)
    # 但為了效能，我們不需要真的展開。我們只要知道: 
    # 當 Return Date 在 [start, end] 之間時，該公司屬於哪個 Rank。
    
    current_year_data['Portfolio_Year'] = year # 這是財報年度
    portfolio_assignments.append(current_year_data[['Company_ID', 'Portfolio_Year', 'Rank']])

df_assignments = pd.concat(portfolio_assignments)

# 3. 合併報酬率
# 這是一個 Many-to-Many 的對應，我們需要一個有效率的方法
# 方法: 為 Return 資料加上 "對應的財報年度" (Portfolio_Year)
# 規則: 如果是 2015/05 ~ 2016/04，對應的財報年度是 2014
# 公式: Portfolio_Year = Ret_Year - 1 (如果月份 >= 5) else Ret_Year - 2

conditions = [
    df_ret['Ret_Month'] >= 5,
    df_ret['Ret_Month'] < 5
]
choices = [
    df_ret['Ret_Year'] - 1,
    df_ret['Ret_Year'] - 2
]
df_ret['Portfolio_Year'] = np.select(conditions, choices)

# 現在可以 Merge 了
print("正在合併財報與報酬率...")
df_final = pd.merge(df_ret, df_assignments, on=['Company_ID', 'Portfolio_Year'], how='inner')

# 4. 計算投資組合績效
print("計算績效中...")

# 計算每個月、每個 Rank 的平均報酬
monthly_portfolio_ret = df_final.groupby(['Date', 'Rank'])['Return'].mean().reset_index()

# 整理成 Pivot Table: Index=Date, Columns=Rank
pivot_ret = monthly_portfolio_ret.pivot(index='Date', columns='Rank', values='Return')
pivot_ret.columns = ['Q1_Low', 'Q2', 'Q3', 'Q4', 'Q5_High']

# 計算多空策略 (Long-Short)
pivot_ret['Long_Short'] = pivot_ret['Q5_High'] - pivot_ret['Q1_Low']

# 5. 輸出統計結果
print("\n" + "="*50)
print("【投資組合策略績效分析】")
print("策略: 每年 5 月買入高 MainSG&A 投資公司 (Q5)，放空低投資公司 (Q1)")
print("樣本期間: ", pivot_ret.index.min().strftime('%Y-%m'), "至", pivot_ret.index.max().strftime('%Y-%m'))
print("="*50)

# 計算年化報酬與檢定
# 將月報酬 * 100 轉為百分比方便閱讀
stats_df = pivot_ret * 100

summary = pd.DataFrame({
    'Mean Return (%)': stats_df.mean(),
    'Std Dev (%)': stats_df.std(),
    't-stat': [stats.ttest_1samp(stats_df[col], 0)[0] for col in stats_df.columns],
    'p-value': [stats.ttest_1samp(stats_df[col], 0)[1] for col in stats_df.columns]
})


summary[['Mean Return (%)', 't-stat', 'p-value']].round(3)




報酬率資料筆數: 408713
正在建構投資組合...
正在合併財報與報酬率...
計算績效中...

【投資組合策略績效分析】
策略: 每年 5 月買入高 MainSG&A 投資公司 (Q5)，放空低投資公司 (Q1)
樣本期間:  2015-05 至 2024-12


/var/folders/yw/h2hzj6ns63l5ybd0x5xs5gr80000gn/T/ipykernel_1968/2373783210.py:84: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_portfolio_ret = df_final.groupby(['Date', 'Rank'])['Return'].mean().reset_index()


,Mean Return (%),t-stat,p-value
Q1_Low,1.151,2.822,0.006
Q2,1.291,2.917,0.004
Q3,1.515,3.185,0.002
Q4,1.460,3.079,0.003
Q5_High,1.452,3.458,0.001
Long_Short,0.301,1.902,0.060


###
加入因子模型（效果不彰）

In [75]:
def analyze_factors():
    print("正在執行因子分析 (Factor Analysis)...")

    # 1. 讀取投資組合報酬率 (Long-Short)
    # 這是先前 analyze_hedge_portfolio.py 產出的檔案 (小數格式)
    try:
        df_port = pd.read_csv('../data/processed/hedge_portfolio_performance.csv')
    except FileNotFoundError:
        print("錯誤: 找不到 '../data/processed/hedge_portfolio_performance.csv'，請先執行 analyze_hedge_portfolio.py")
        return

    # 處理日期格式
    df_port['Date'] = pd.to_datetime(df_port['Date'])
    df_port['YearMonth'] = df_port['Date'].dt.strftime('%Y%m')
    
    # 將報酬率從小數轉為百分比 (因為因子資料是百分比)
    # 我們主要分析 Long_Short 欄位
    if 'Long_Short' not in df_port.columns:
        print("錯誤: 投資組合資料中缺少 'Long_Short' 欄位")
        return
        
    df_port['Long_Short_Pct'] = df_port['Long_Short'] * 100

    # 2. 讀取 8factors.csv
    try:
        # 假設該檔案在當前目錄
        df_factors = pd.read_csv('../data/raw/8factors.csv')
    except FileNotFoundError:
        print("錯誤: 找不到 '../data/raw/8factors.csv'")
        return

    # 處理因子資料
    # 欄位: 證券代碼,年月,市場風險溢酬,規模溢酬 (5因子),淨值市價比溢酬,盈利能力因子,投資因子,無風險利率,動能因子,短期反轉因子
    df_factors['YearMonth'] = df_factors['年月'].astype(str)
    
    # 重新命名欄位以便操作
    rename_dict = {
        '市場風險溢酬': 'Mkt_RF',
        '規模溢酬 (5因子)': 'SMB',
        '淨值市價比溢酬': 'HML',
        '盈利能力因子': 'RMW',
        '投資因子': 'CMA',
        '動能因子': 'MOM',
        '短期反轉因子': 'STR',
        '無風險利率': 'RF'
    }
    df_factors = df_factors.rename(columns=rename_dict)

    # 3. 合併資料
    df_merged = pd.merge(df_port, df_factors, on='YearMonth', how='inner')
    
    if len(df_merged) == 0:
        print("警告: 合併後無資料，請檢查日期格式是否一致")
        return
    
    print(f"合併後樣本期間: {df_merged['YearMonth'].min()} 至 {df_merged['YearMonth'].max()} (共 {len(df_merged)} 個月)")

    # 4. 定義迴歸模型
    # 依變數: Long_Short_Pct
    # 自變數: 各種因子組合
    
    # 因子清單 (不含 RF)
    factors_all = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM', 'STR']
    
    models = {
        'CAPM (1 Factor)': ['Mkt_RF'],
        'FF3 (3 Factors)': ['Mkt_RF', 'SMB', 'HML'],
        'Carhart (4 Factors)': ['Mkt_RF', 'SMB', 'HML', 'MOM'],
        'FF5 (5 Factors)': ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA'],
        'FF5 + MOM (6 Factors)': ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM'],
        'All (7 Factors)': ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM', 'STR']
    }

    results_summary = []

    y = df_merged['Long_Short_Pct']

    print("\n" + "="*80)
    print(f"{ 'Model':<25} | {'Alpha (%)':<10} | {'t-stat':<8} | {'p-value':<8} | {'Adj. R2':<8}")
    print("-" * 80)

    for model_name, factor_list in models.items():
        X = df_merged[factor_list]
        X = sm.add_constant(X) # 加上截距項 (Alpha) 
        
        model = sm.OLS(y, X).fit()
        
        alpha = model.params['const']
        t_stat = model.tvalues['const']
        p_val = model.pvalues['const']
        adj_r2 = model.rsquared_adj
        
        # 標記顯著性
        sig = ""
        if p_val < 0.01: sig = "***"
        elif p_val < 0.05: sig = "**"
        elif p_val < 0.1: sig = "*"
        
        print(f"{model_name:<25} | {alpha:7.3f} {sig:<3} | {t_stat:7.3f}  | {p_val:7.3f}  | {adj_r2:7.3f}")
        
        results_summary.append({
            'Model': model_name,
            'Alpha': alpha,
            't-stat': t_stat,
            'p-value': p_val,
            'Adj_R2': adj_r2,
            'Factors': ", ".join(factor_list)
        })

    print("-" * 80)
    print("註: Alpha 為月報酬，*** p<0.01, ** p<0.05, * p<0.1")
    
    # 5. 詳細輸出最後一個模型 (All Factors) 的係數
    print("\n【完整模型 (7 Factors) 係數分析】")
    final_model_factors = models['All (7 Factors)']
    X_final = sm.add_constant(df_merged[final_model_factors])
    final_model = sm.OLS(y, X_final).fit()
    print(final_model.summary())

if __name__ == "__main__":
    analyze_factors()


正在執行因子分析 (Factor Analysis)...
合併後樣本期間: 201505 至 202412 (共 116 個月)

Model                     | Alpha (%)  | t-stat   | p-value  | Adj. R2 
--------------------------------------------------------------------------------
CAPM (1 Factor)           |   0.518     |   1.652  |   0.101  |   0.366
FF3 (3 Factors)           |   0.244     |   1.140  |   0.257  |   0.709
Carhart (4 Factors)       |   0.243     |   1.067  |   0.288  |   0.707
FF5 (5 Factors)           |   0.270     |   1.240  |   0.217  |   0.734
FF5 + MOM (6 Factors)     |   0.273     |   1.171  |   0.244  |   0.731
All (7 Factors)           |   0.300     |   1.308  |   0.194  |   0.741
--------------------------------------------------------------------------------
註: Alpha 為月報酬，*** p<0.01, ** p<0.05, * p<0.1

【完整模型 (7 Factors) 係數分析】
                            OLS Regression Results                            
Dep. Variable:         Long_Short_Pct   R-squared:                       0.757
Model:                            OLS  

###
後續進行Equation 6的實證（Future Earnings Volatility and Uncertainty of Future Benefits)



In [ ]:
# 建構依變數: Future Earnings Volatility (t to t+3)
# 計算滾動 4 年的標準差 (包含當年)
df_merged = df_merged.sort_values(by=['Company_ID', 'Year'])

# 使用 rolling window 計算標準差
# rolling(4) 會包含當前行與前3行。但論文定義是 t 到 t+3。
# 技巧：使用 shift 把未來的資料移到當前行，或者使用反向 rolling (pandas 比較難直接做)。
# 替代方法：Shift t+1, t+2, t+3 到當前行，然後橫向計算 std。

df_merged['Earnings_t0'] = df_merged['Scaled_Earnings']
df_merged['Earnings_t1'] = df_merged.groupby('Company_ID')['Scaled_Earnings'].shift(-1)
df_merged['Earnings_t2'] = df_merged.groupby('Company_ID')['Scaled_Earnings'].shift(-2)
df_merged['Earnings_t3'] = df_merged.groupby('Company_ID')['Scaled_Earnings'].shift(-3)

# 計算橫向標準差 (需至少 3 年有資料才計算，避免誤差太大)
earnings_cols = ['Earnings_t0', 'Earnings_t1', 'Earnings_t2', 'Earnings_t3']
df_merged['Future_Earnings_Vol'] = df_merged[earnings_cols].std(axis=1, ddof=1) # ddof=1 for sample std

# 6. 建構控制變數
df_merged['Size'] = np.log(df_merged['MV'])
df_merged['Leverage'] = df_merged['Total_Liabilities'] / df_merged['Total_Assets']
df_merged['Loss_Dummy'] = (df_merged['Net_Income'] < 0).astype(int)
df_merged['Current_Earnings_Growth'] = df_merged['Current_Earnings_Growth'].fillna(0)

# 7. 準備迴歸資料
cols_to_use = ['Future_Earnings_Vol', 'Scaled_RD', 'Investment_MainSGA', 
               'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
               'Size', 'Current_Earnings_Growth', 'Loss_Dummy', 'Leverage', 
               'Company_ID', 'Year', 'Industry_Group']

df_reg = df_merged.dropna(subset=cols_to_use).copy()

# === Step A: Winsorization (1% & 99%) ===
print("正在進行 Winsorization (1%, 99%):..")
vars_to_winsorize = ['Future_Earnings_Vol', 'Scaled_RD', 'Investment_MainSGA', 
                     'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
                     'Size', 'Current_Earnings_Growth', 'Leverage']

for col in vars_to_winsorize:
    df_reg[col] = winsorize(df_reg[col], limits=[0.01, 0.01])

# === Step B: Industry-Year Standardization (Z-Score) ===
print("正在進行 Industry-Year Standardization...")
invest_vars = ['Scaled_RD', 'Investment_MainSGA', 'Scaled_PPE_Capex', 'Scaled_Intan_Capex']

for col in invest_vars:
    new_col = f'Std_{col}'
    grouped = df_reg.groupby(['Industry_Group', 'Year'])[col]
    mean = grouped.transform('mean')
    std = grouped.transform('std')
    df_reg[new_col] = (df_reg[col] - mean) / std
    df_reg[new_col] = df_reg[new_col].fillna(0)

# Company & Year Code for Clustering
df_reg['Company_Code'] = df_reg['Company_ID'].astype('category').cat.codes
df_reg['Year_Code'] = df_reg['Year'].astype(int)
groups = df_reg[['Company_Code', 'Year_Code']].values

print(f"\n分析樣本數: {len(df_reg)}")

# Model A1: Future Earnings Volatility (Raw Values)
print("\n=== Equation 6 (A1): Future Earnings Volatility (Raw Values) ===")
formula_a1 = 'Future_Earnings_Vol ~ Scaled_RD + Investment_MainSGA + Scaled_PPE_Capex + Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_a1 = smf.ols(formula_a1, data=df_reg)
res_a1 = model_a1.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_a1.summary())

# Model A2: Future Earnings Volatility (Standardized Values)
print("\n=== Equation 6 (A2): Future Earnings Volatility (Standardized Inv) ===")
formula_a2 = 'Future_Earnings_Vol ~ Std_Scaled_RD + Std_Investment_MainSGA + Std_Scaled_PPE_Capex + Std_Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_a2 = smf.ols(formula_a2, data=df_reg)
res_a2 = model_a2.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_a2.summary())

# 整理係數比較 (Table 8 Style)
print("\n" + "="*80)
print("【Table 8 Replication】: Uncertainty of Future Benefits")
print("Dependent Variable: Future Earnings Volatility (Std Dev t~t+3)")
print("="*80)
print(f"{ '':<20} | { 'Raw Coef':<15} | { 'Std Coef':<15} | { 't-stat (Std)':<10}")
print("-" * 80)

rows = [
    ('R&D', 'Scaled_RD', 'Std_Scaled_RD'),
    ('MainSG&A Inv', 'Investment_MainSGA', 'Std_Investment_MainSGA'),
    ('PPE Capex', 'Scaled_PPE_Capex', 'Std_Scaled_PPE_Capex'),
    ('Intan Capex', 'Scaled_Intan_Capex', 'Std_Scaled_Intan_Capex'),
    ('Leverage', 'Leverage', 'Leverage')
]

for label, raw_var, std_var in rows:
    # Raw Model
    raw_coef = res_a1.params[raw_var]
    
    # Std Model
    var_a2 = std_var if std_var in res_a2.params else raw_var
    std_coef = res_a2.params[var_a2]
    t_stat = res_a2.tvalues[var_a2]
    
    # Sig
    sig = "***" if res_a2.pvalues[var_a2] < 0.01 else "**" if res_a2.pvalues[var_a2] < 0.05 else "*" if res_a2.pvalues[var_a2] < 0.1 else ""

    print(f"{label:<20} | {raw_coef:>8.4f}        | {std_coef:>8.4f}{sig:<3} | {t_stat:>6.2f}")

print("-" * 80)
print("Interpretation: Positive coef = Higher Risk (More Volatility)")


正在進行 Winsorization (1%, 99%):..
正在進行 Industry-Year Standardization...

分析樣本數: 16538

=== Equation 6 (A1): Future Earnings Volatility (Raw Values) ===
                             OLS Regression Results                            
Dep. Variable:     Future_Earnings_Vol   R-squared:                       0.161
Model:                             OLS   Adj. R-squared:                  0.160
Method:                  Least Squares   F-statistic:                     68.08
Date:                 Fri, 12 Dec 2025   Prob (F-statistic):           3.82e-07
Time:                         03:42:30   Log-Likelihood:                 31009.
No. Observations:                16538   AIC:                        -6.200e+04
Df Residuals:                    16529   BIC:                        -6.193e+04
Df Model:                            8                                         
Covariance Type:               cluster                                         
                              coef    std err     

###
進一步比較Maintenance SGA成分與Investment SGA

如下表所示，Standardized Investment MainSG&A 的係數顯著為正，且其影響力 (Coefficient Magnitude) 僅次於 R&D，並顯著高於Capex。這證明了市場高度認可 MainSG&A 投資的價值。


同時，Investment 的係數顯著大於 Maintenance

In [72]:
# === Step A: Winsorization (1% & 99%) on RAW values ===
print("正在進行 Winsorization (1%, 99% от...")
vars_to_winsorize = ['Tobins_Q', 'Scaled_RD', 'Investment_MainSGA', 'Maintenance_MainSGA',
                     'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
                     'Size', 'Current_Earnings_Growth', 'Leverage']

for col in vars_to_winsorize:
    df_reg[col] = winsorize(df_reg[col], limits=[0.01, 0.01])

# === Step B: Industry-Year Standardization (Z-Score) ===
print("正在進行 Industry-Year Standardization...")
# 我們也要標準化 Maintenance_MainSGA 以便公平比較
invest_vars = ['Scaled_RD', 'Investment_MainSGA', 'Maintenance_MainSGA', 'Scaled_PPE_Capex', 'Scaled_Intan_Capex']

for col in invest_vars:
    new_col = f'Std_{col}'
    grouped = df_reg.groupby(['Industry_Group', 'Year'])[col]
    mean = grouped.transform('mean')
    std = grouped.transform('std')
    df_reg[new_col] = (df_reg[col] - mean) / std
    df_reg[new_col] = df_reg[new_col].fillna(0)

# Company & Year Code for Clustering
df_reg['Company_Code'] = df_reg['Company_ID'].astype('category').cat.codes
df_reg['Year_Code'] = df_reg['Year'].astype(int)
groups = df_reg[['Company_Code', 'Year_Code']].values

print(f"\n分析樣本數: {len(df_reg)}")

# Model 1: Investment Component
print("\n=== Model 1: Investment Component (Tobin's Q) ===")
formula_inv = 'Tobins_Q ~ Std_Scaled_RD + Std_Investment_MainSGA + Std_Scaled_PPE_Capex + Std_Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_inv = smf.ols(formula_inv, data=df_reg)
res_inv = model_inv.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_inv.summary())

# Model 2: Maintenance Component
print("\n=== Model 2: Maintenance Component (Tobin's Q) ===")
formula_maint = 'Tobins_Q ~ Std_Scaled_RD + Std_Maintenance_MainSGA + Std_Scaled_PPE_Capex + Std_Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
model_maint = smf.ols(formula_maint, data=df_reg)
res_maint = model_maint.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(res_maint.summary())

# 整理係數比較 (Validity Test)
print("\n" + "="*80)
print("【Validity Test】: Investment vs. Maintenance Component")
print("Dependent Variable: Tobin's Q")
print("="*80)
print(f"{ '':<25} | { 'Investment Model':<20} | { 'Maintenance Model':<20}")
print("-" * 80)

# Extract coefficients and stats
# Investment Model
inv_coef = res_inv.params['Std_Investment_MainSGA']
inv_t = res_inv.tvalues['Std_Investment_MainSGA']
inv_p = res_inv.pvalues['Std_Investment_MainSGA']

# Maintenance Model
maint_coef = res_maint.params['Std_Maintenance_MainSGA']
maint_t = res_maint.tvalues['Std_Maintenance_MainSGA']
maint_p = res_maint.pvalues['Std_Maintenance_MainSGA']

def get_sig(p):
    return "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.1 else ""

print(f"{'Main Coefficient':<25} | {inv_coef:>8.4f}{get_sig(inv_p):<3} (t={inv_t:.2f}) | {maint_coef:>8.4f}{get_sig(maint_p):<3} (t={maint_t:.2f})")
print("-" * 80)
print("Hypothesis: Investment Coef > 0 (Significant), Maintenance Coef ~ 0 (Insignificant)")



正在進行 Winsorization (1%, 99% от...
正在進行 Industry-Year Standardization...

分析樣本數: 16538

=== Model 1: Investment Component (Tobin's Q) ===
                            OLS Regression Results                            
Dep. Variable:               Tobins_Q   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     48.53
Date:                Fri, 12 Dec 2025   Prob (F-statistic):           1.68e-06
Time:                        03:39:59   Log-Likelihood:                -21472.
No. Observations:               16538   AIC:                         4.296e+04
Df Residuals:                   16529   BIC:                         4.303e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      

延伸分析：產業差異 電子業 vs 非電子業 


考量台灣股市以電子業為主導，進一步將樣本分為「電子業」與「非電子業」進行次樣本分析。

電子業：MainSG&A 投資效益顯著較高 (Coef = 0.195)，反映知識經濟特性。


非電子業：效益較低但仍顯著 (Coef = 0.109)。

規模：大公司與小公司均能從 MainSG&A 投資中獲益。


In [ ]:
# 1. 電子業 vs 非電子業
# 假設 M23, M24, M25, M26, M27, M28 等為電子相關 (依據 TEJ 常用代碼)
# 電子零組件(M23), 半導體(M24), 光電(M25), 電腦週邊(M26), 通信網路(M27), 電子通路(M28), 資訊服務(M29)
electronic_codes = ['M23', 'M24', 'M25', 'M26', 'M27', 'M28', 'M29']
df_merged['Is_Electronics'] = df_merged['Industry_Group'].isin(electronic_codes)

# 2. 公司規模 (Size) - Proxy for Lifecycle (Small ~ Young/Growth, Large ~ Mature)
# 使用全樣本中位數切分
size_median = df_merged['Size'].median()
df_merged['Is_Large'] = df_merged['Size'] > size_median

# Data Cleaning
cols = ['Tobins_Q', 'Scaled_RD', 'Investment_MainSGA', 'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
        'Size', 'Current_Earnings_Growth', 'Loss_Dummy', 'Leverage', 
        'Industry_Group', 'Year', 'Company_ID', 'Is_Electronics', 'Is_Large']
df_reg = df_merged.dropna(subset=cols).copy()

# Winsorization
vars_win = ['Tobins_Q', 'Scaled_RD', 'Investment_MainSGA', 'Scaled_PPE_Capex', 'Scaled_Intan_Capex', 
            'Size', 'Current_Earnings_Growth', 'Leverage']
for col in vars_win:
    df_reg[col] = winsorize(df_reg[col], limits=[0.01, 0.01])

# Standardization (Z-Score by Industry-Year)
invest_vars = ['Scaled_RD', 'Investment_MainSGA', 'Scaled_PPE_Capex', 'Scaled_Intan_Capex']
for col in invest_vars:
    new_col = f'Std_{col}'
    grouped = df_reg.groupby(['Industry_Group', 'Year'])[col]
    mean = grouped.transform('mean')
    std = grouped.transform('std')
    df_reg[new_col] = (df_reg[col] - mean) / std
    df_reg[new_col] = df_reg[new_col].fillna(0)

# Clustering Setup
df_reg['Company_Code'] = df_reg['Company_ID'].astype('category').cat.codes
df_reg['Year_Code'] = df_reg['Year'].astype(int)

# ==========================================
# 執行回歸函數
# ==========================================
def run_regression(data, label):
    # 如果樣本太少，避免 Clustering 錯誤
    if len(data) < 100:
        return "N/A", 0, 0
        
    groups = data[['Company_Code', 'Year_Code']].values
    formula = 'Tobins_Q ~ Std_Scaled_RD + Std_Investment_MainSGA + Std_Scaled_PPE_Capex + Std_Scaled_Intan_Capex + Size + Current_Earnings_Growth + Loss_Dummy + Leverage'
    model = smf.ols(formula, data=data)
    
    try:
        res = model.fit(cov_type='cluster', cov_kwds={'groups': groups})
    except:
        # Fallback to robust SE if clustering fails (e.g. too few groups)
        res = model.fit(cov_type='HC1')
    
    coef = res.params['Std_Investment_MainSGA']
    t_stat = res.tvalues['Std_Investment_MainSGA']
    p_val = res.pvalues['Std_Investment_MainSGA']
    obs = int(res.nobs)
    adj_r2 = res.rsquared_adj
    
    sig = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
    return f"{coef:.4f}{sig} (t={t_stat:.2f})", obs, adj_r2

print("\n" + "="*30)
print("Extended Analysis Results")
print("="*30)

# 1. Electronics vs Non-Electronics
res_elec = run_regression(df_reg[df_reg['Is_Electronics'] == True], "Electronics")
res_non = run_regression(df_reg[df_reg['Is_Electronics'] == False], "Non-Electronics")

# 2. Small vs Large
res_small = run_regression(df_reg[df_reg['Is_Large'] == False], "Small Firms")
res_large = run_regression(df_reg[df_reg['Is_Large'] == True], "Large Firms")

# Full Sample
res_all = run_regression(df_reg, "Full Sample")

# Print Markdown Table
print("\n### Sub-sample Analysis: Impact of MainSG&A Investment on Tobin's Q\n")
print("| Sample Group | MainSG&A Coef (Std) | Obs | Adj. R2 |")
print(f"| **Full Sample** | {res_all[0]} | {res_all[1]} | {res_all[2]:.3f} |")
print(f"| Electronics | {res_elec[0]} | {res_elec[1]} | {res_elec[2]:.3f} |")
print(f"| Non-Electronics | {res_non[0]} | {res_non[1]} | {res_non[2]:.3f} |")
print(f"| Small Firms (Size < Med) | {res_small[0]} | {res_small[1]} | {res_small[2]:.3f} |")
print(f"| Large Firms (Size > Med) | {res_large[0]} | {res_large[1]} | {res_large[2]:.3f} |")
print("\n*Note: Coeffs are standardized. t-stats in parentheses are clustered by firm & year. *** p<0.01, ** p<0.05, * p<0.1*")



Extended Analysis Results

### Sub-sample Analysis: Impact of MainSG&A Investment on Tobin's Q

| Sample Group | MainSG&A Coef (Std) | Obs | Adj. R2 |
| **Full Sample** | 0.1629*** (t=7.50) | 18372 | 0.267 |
| Electronics | 0.1947*** (t=6.72) | 11501 | 0.268 |
| Non-Electronics | 0.1085*** (t=3.78) | 6871 | 0.282 |
| Small Firms (Size < Med) | 0.1669*** (t=7.00) | 9186 | 0.180 |
| Large Firms (Size > Med) | 0.1840*** (t=5.38) | 9186 | 0.309 |

*Note: Coeffs are standardized. t-stats in parentheses are clustered by firm & year. *** p<0.01, ** p<0.05, * p<0.1*
